In [1]:
#To extract page numbers(pb), div2 label, and catnum from Misc XML file and save as csv.

#Extract div3 when there is no catnum within each div2 labels

#Excludes Indexes
#To ignore digits from <add rend="del"> and <add rend="pencil"> which occurs once with catnum 2082
#from <catnum>, remove: -; ____; ?; .; and a tick in front of a number.
#Excludes non-Sloane cat entries.

from lxml import etree
import re
import csv

In [2]:
path = '/Users/deborahleem/Documents/PhD/SloaneDBwork19/Miscellanies5Ah_MASTER_fixed_whitespaceDL.xml'
doc = etree.parse(open(path))
root = doc.getroot()

In [3]:
def remove_element(el):
    parent = el.getparent()
    if el.tail:
        prev = el.getprevious()
        if prev:
            prev.tail = (prev.tail or '') + el.tail
        else:
            parent.text = (parent.text or '') + el.tail
    parent.remove(el)

# Remove every <add rend="del"> and <add rend="pencil"> from document before the rest of analysis
for e in list(root.iterfind('.//{*}add')):
    if ('rend' in e.attrib) and (e.attrib['rend'] == 'del' or e.attrib['rend'] == 'pencil'):
        #print('removing', etree.tostring(e, pretty_print=True))
        if e.text == '-1861' or e.text == '2094.' or e.text == '-382.':
            continue
        remove_element(e)


/Users/deborahleem/scripts/venvs/deb_1.0/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  """


In [4]:
#Catnums that need to be excluded as they are not part of Sloane
#<pb xml:id="EPF108491027v"/>, Miscellanies, 2108, 2109, 2110, 2111
#<pb xml:id="EPF108491177v"/>, Antiquities, 1,2
#<pb xml:id="EPF108491178v"/>, Antiquities, 1,2,3,4
#<pb xml:id="EPF108491179v"/>, Antiquities, 4,5,6,7,8,1
#<pb xml:id="EPF108491180v"/>, Antiquities, 2,3
#<pb xml:id="EPF108491181v"/>, Antiquities, 4,5
#<pb xml:id="EPF108491182v"/>, Antiquities, 1,2,3,4,5
#<pb xml:id="EPF108491183v"/>, Antiquities, 6,7,8,9,10,11,12
#<pb xml:id="EPF108491184v"/>, Antiquities, 13,14,1,2,3,4,5,1
#<pb xml:id="EPF108491186v" type="Bronzes"/>, Antiquities, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
#<pb xml:id="EPF108491187v" type="Bronzes"/>, Antiquities, 20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
#<pb xml:id="EPF108491188v"/>, Antiquities, 40,41,42,43,44,45,46,47,48,1,2,3,4,5,6,7,8,9
#<pb xml:id="EPF108491189v"/>, Antiquities, 10,11,12,1,2,3,4,5,6
#<pb xml:id="EPF108491190v"/>, Antiquities, 7,8,9,10,11

In [5]:
excluded = {
  "EPF108491027v": ["2108","2109","2110","2111"], # "Miscellanies"
  "EPF108491177v": ["1","2"], # "Antiquities"
  "EPF108491178v": ["1","2","3","4"], # "Antiquities"
  "EPF108491179v": ["4","5","6","7","8","1"], # "Antiquities"
  "EPF108491180v": ["2","3"], # "Antiquities"
  "EPF108491181v": ["4","5"], # "Antiquities"
  "EPF108491182v": ["1","2","3","4","5"], # "Antiquities"
  "EPF108491183v": ["6","7","8","9","10","11","12"], # "Antiquities"
  "EPF108491184v": ["13","14","1","2","3","4","5","1"], # "Antiquities"
  "EPF108491186v": ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"], # "Antiquities"
  "EPF108491187v": ["20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39"], # "Antiquities"
  "EPF108491188v": ["40","41","42","43","44","45","46","47","48","1","2","3","4","5","6","7","8","9"], # "Antiquities"
  "EPF108491189v": ["10","11","12","1","2","3","4","5","6"], # "Antiquities"
  "EPF108491190v": ["7","8","9","10","11"], # "Antiquities"
}


In [6]:
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return ''.join(label.itertext()).strip()
            return
        element = element.getparent()

In [7]:
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
#         print(element.tag)
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()
                
# for catnum in root.iterfind('.//{*}catnum'):
#     print(find_pb_xml_id(catnum))


In [ ]:
#metamark not consistenly used.

In [8]:
def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

results = []
for catnum in root.iterfind('.//{*}catnum'):
    text = "".join(catnum.itertext())
    text = cleanup_catnum(text)
    label = find_div2_label(catnum)
    if label.startswith('Index'):
        continue
    pb_xml_id = find_pb_xml_id(catnum)
    if pb_xml_id in excluded:
        if text in excluded[pb_xml_id]:
#             print('exclude', pb_xml_id, text)
            continue
#         else:
#             print('do not exclude', pb_xml_id, text)
    
    results.append([text, label, pb_xml_id])
    #print(etree.tostring(catnum))
    #print([text, label])

In [9]:
# Find div3s without catnums inside, not in index
#There are many Div3 without catnum on recto so only retrieve ones on verso
for div3 in root.iterfind('.//{*}div3'):
    catnums_inside = list(div3.iterfind('.//{*}catnum'))
    if len(catnums_inside) > 0:
        continue
    label = find_div2_label(div3)
    if label.startswith('Index'):
        continue
    pb_xml_id = find_pb_xml_id(div3)
    if pb_xml_id in excluded:
         continue
    if pb_xml_id.endswith('v'):
        results.append(['no catnum', label, pb_xml_id])
#     print(div3)
#     print(etree.tostring(div3))
#     break


In [10]:
# results0 = results

In [11]:
#Showing all changes.
# [(results0[i], results[i]) for i in range(len(results)) if results0[i] != results[i]]

In [12]:
# All entries that are not just digits and start
[[x, results.index(x)] for x in results if not x[0].replace('*', '').isdigit()]

[[['No 1', 'Miscellanies.', 'EPF108491009v'], 238],
 [['1933 No 7', 'Miscellanies.', 'EPF108491010v'], 259],
 [['1933 No 8', 'Miscellanies.', 'EPF108491010v'], 260],
 [['1933 No 9', 'Miscellanies.', 'EPF108491010v'], 261],
 [['1933 No 10', 'Miscellanies.', 'EPF108491010v'], 262],
 [['1933 No 11', 'Miscellanies.', 'EPF108491010v'], 263],
 [['1933 No 12', 'Miscellanies.', 'EPF108491010v'], 264],
 [['1933 No 13', 'Miscellanies.', 'EPF108491010v'], 265],
 [['1933 No 14', 'Miscellanies.', 'EPF108491010v'], 266],
 [['1933 No 15', 'Miscellanies.', 'EPF108491010v'], 267],
 [['1933 No 16', 'Miscellanies.', 'EPF108491010v'], 268],
 [['1933 No 17', 'Miscellanies.', 'EPF108491010v'], 269],
 [['1933 No 18', 'Miscellanies.', 'EPF108491010v'], 270],
 [['1933 No 20', 'Miscellanies.', 'EPF108491010v'], 271],
 [['1933 No 21', 'Miscellanies.', 'EPF108491010v'], 272],
 [['1933 No 22', 'Miscellanies.', 'EPF108491010v'], 273],
 [['1933 No 25', 'Miscellanies.', 'EPF108491010v'], 274],
 [['1933 No 26', 'Misce

In [13]:
with open('all_catnums_and_their_div2_labels.csv', 'w') as file:
    writer = csv.writer(file)
    for catnum, label, pageid in results:
        writer.writerow([catnum, label, pageid])

In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('all_catnums_and_their_div2_labels.csv', header=None, names=['catnum', 'div2_label', 'page_nu'])

In [16]:
df.head()

,catnum,div2_label,page_nu
0,1799,Miscellanies.,EPF108491002v
1,1800,Miscellanies.,EPF108491002v
2,1801,Miscellanies.,EPF108491002v
3,1802,Miscellanies.,EPF108491002v
4,1803,Miscellanies.,EPF108491002v


In [17]:

# is catnum empty?
df[df.catnum == None]
df[df.catnum.isnull()]
df[df.catnum == '']

,catnum,div2_label,page_nu


In [18]:
df[df.div2_label.isnull()]

,catnum,div2_label,page_nu


In [19]:
df[df.div2_label == '']

,catnum,div2_label,page_nu


In [20]:
df.sort_values('div2_label', ascending=False)

,catnum,div2_label,page_nu
3910,316,Pictures &c.,EPF108491264
3866,272,Pictures &c.,EPF108491261
3876,282,Pictures &c.,EPF108491262
3875,281,Pictures &c.,EPF108491262
3874,280,Pictures &c.,EPF108491262
...,...,...,...
4409,105,Agate cups botles spoons &c.,EPF108491314
4408,104,Agate cups botles spoons &c.,EPF108491314
4407,103,Agate cups botles spoons &c.,EPF108491314
4406,102,Agate cups botles spoons &c.,EPF108491314


In [36]:
#total number of catnum = 4608
len(df)

4629

In [22]:
len(df.div2_label.unique())

7

In [37]:
df.div2_label.unique()

array(['Miscellanies.', 'Antiquities.', 'Impressions of Seals &c.',
       'Pictures &c.', 'Mathematicall instruments &c',
       'Agate handles &c.', 'Agate cups botles spoons &c.'], dtype=object)

In [38]:
#unique number of pages = 369 including Index. Without 382

len(df.page_nu.unique())

382

In [39]:

misc = df[df.div2_label == 'Miscellanies.']

In [40]:
len(misc.catnum)

2181

In [41]:
len(misc.catnum.unique())

2148

In [42]:
misc[misc.catnum.duplicated(keep=False)].sort_values('catnum')

,catnum,div2_label,page_nu
1382,1025,Miscellanies.,EPF108491107v
1385,1025,Miscellanies.,EPF108491108
1434,1073,Miscellanies.,EPF108491111v
1433,1073,Miscellanies.,EPF108491111
1818,1455,Miscellanies.,EPF108491133
1805,1455,Miscellanies.,EPF108491132v
1856,1492,Miscellanies.,EPF108491135
1855,1492,Miscellanies.,EPF108491135
1921,1556,Miscellanies.,EPF108491138
1920,1556,Miscellanies.,EPF108491138


In [43]:
misc[misc.catnum.duplicated(keep=False)].sort_values('catnum').to_csv("dupicate_catnum_each_div2.csv")

In [44]:
antq = df[df.div2_label == 'Antiquities.']

In [45]:
len(antq.catnum)

1175

In [46]:
len(antq.catnum.unique())

1153

In [47]:
antq[antq.catnum.duplicated(keep=False)].sort_values('catnum')

,catnum,div2_label,page_nu
2673,1048,Antiquities.,EPF108491170v
2674,1048,Antiquities.,EPF108491170v
2875,252,Antiquities.,EPF108491180a
2849,252,Antiquities.,EPF108491178
2850,253,Antiquities.,EPF108491179
2876,253,Antiquities.,EPF108491180a
2851,254,Antiquities.,EPF108491179
2877,254,Antiquities.,EPF108491180a
2878,255,Antiquities.,EPF108491180a
2852,255,Antiquities.,EPF108491179


In [33]:
cups = df[df.div2_label == 'Agate cups botles spoons &c.']

In [34]:
len(cups.catnum)

303

In [35]:
len(cups.catnum.unique())

303

In [49]:
seals = df[df.div2_label == 'Impressions of Seals &c.']

In [50]:
len(seals.catnum)

260

In [51]:
len(seals.catnum.unique())

260

In [55]:
pics = df[df.div2_label == 'Pictures &c.']

In [56]:
len(pics.catnum)

413

In [57]:
len(pics.catnum.unique())

411

In [58]:
pics[pics.catnum.duplicated(keep=False)].sort_values('catnum')

,catnum,div2_label,page_nu
3962,368,Pictures &c.,EPF108491267
3963,368,Pictures &c.,EPF108491268
3978,382,Pictures &c.,EPF108491268
3987,382,Pictures &c.,EPF108491269


In [ ]:
#to get text from div3: .join(div3.itertext()).strip()